In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
os.chdir("..")

In [2]:
# testing CIFAR 10 for speed with fp16 or fp32
import numpy as np
import tensorflow as tf
# tf.enable_eager_execution()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = False  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)

sess = tf.Session(config=config)
tf.keras.backend.set_session(sess)

In [3]:
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
x_train = x_train / x_train.max()
x_test = x_test / x_test.max()

In [4]:
def fp16_loss_generator(loss_fn, multiplier=100):
    def custom_loss(yTrue, yPred):
        return multiplier * loss_fn(yTrue, yPred)
    return custom_loss

In [5]:
# utils
from layers.BatchNormalization16 import BatchNormalizationF16

def _cast_layer(datatype):
    return tf.keras.layers.Lambda(lambda x: tf.cast(x, datatype))
def _conv_bn_leaky(x, filters, kernel_size, alpha=0.3, dtype=tf.float32, conv_args={}, bn_args={}):
    x = tf.keras.layers.Conv2D(filters=filters,
                               kernel_size=kernel_size,
                               **conv_args)(x)
    x = _cast_layer(tf.float32)(x)
#     x = tf.keras.layers.BatchNormalization(**bn_args)(x)
    x = BatchNormalizationF16(**bn_args)(x)
    x = _cast_layer(dtype)(x)

    return tf.keras.layers.LeakyReLU(alpha=alpha)(x)

def _projection_shortcut(x, filters):
    return tf.keras.layers.Conv2D(filters=filters, kernel_size=[1,1])(x)

In [6]:
def build_model(dtype, weight_decay=1e-4):

    inputs = tf.keras.layers.Input(shape=[32, 32, 3], dtype=dtype)
    net = tf.keras.layers.Conv2D(filters=32,
                                 kernel_size=[3, 3],
                                 padding="same",
                                 activation=None)(inputs)
    # net = tf.keras.layers.BatchNormalization()(net)
    net = tf.keras.layers.LeakyReLU()(net)

    residual = _projection_shortcut(net, 32)

    net = _conv_bn_leaky(net, 32, [3, 3], dtype=dtype, conv_args={"padding": "same", "kernel_regularizer": tf.keras.regularizers.l2(weight_decay)})
    net = _conv_bn_leaky(net, 32, [3, 3], dtype=dtype, conv_args={"padding": "same", "kernel_regularizer": tf.keras.regularizers.l2(weight_decay)})
    net = tf.keras.layers.Add()([residual, net])
    net = tf.keras.layers.MaxPool2D(pool_size=[2,2],
                                    strides=[2,2])(net)

    residual = _projection_shortcut(net, 64)

    net = _conv_bn_leaky(net, 64, [3, 3], dtype=dtype, conv_args={"padding": "same", "kernel_regularizer": tf.keras.regularizers.l2(weight_decay)})
    net = _conv_bn_leaky(net, 64, [3, 3], dtype=dtype, conv_args={"padding": "same", "kernel_regularizer": tf.keras.regularizers.l2(weight_decay)})
    net = tf.keras.layers.Add()([residual, net])
    net = tf.keras.layers.MaxPool2D(pool_size=[2,2],
                                    strides=[2,2])(net)

    residual = _projection_shortcut(net, 128)
    net = _conv_bn_leaky(net, 128, [3, 3], dtype=dtype, conv_args={"padding": "same", "kernel_regularizer": tf.keras.regularizers.l2(weight_decay)})
    net = _conv_bn_leaky(net, 128, [3, 3], dtype=dtype, conv_args={"padding": "same", "kernel_regularizer": tf.keras.regularizers.l2(weight_decay)})
    net = tf.keras.layers.Add()([residual, net])
    net = tf.keras.layers.MaxPool2D(pool_size=[2,2],
                                    strides=[2,2])(net)

    residual = _projection_shortcut(net, 256)
    net = _conv_bn_leaky(net, 256, [3, 3], dtype=dtype, conv_args={"padding": "same", "kernel_regularizer": tf.keras.regularizers.l2(weight_decay)})
    net = _conv_bn_leaky(net, 256, [3, 3], dtype=dtype, conv_args={"padding": "same", "kernel_regularizer": tf.keras.regularizers.l2(weight_decay)})
    net = tf.keras.layers.Add()([residual, net])

    net = tf.keras.layers.GlobalAveragePooling2D()(net)

    net = tf.keras.layers.Dense(512, activation = tf.nn.leaky_relu)(net)

    logits = tf.keras.layers.Dense(10, activation = tf.nn.softmax)(net)
    
    return tf.keras.models.Model(inputs=inputs, outputs=logits)

In [10]:
tf.keras.backend.set_floatx('float16')
tf.keras.backend.set_epsilon(1e-4)
dtype = tf.float16

desired_learning_rate = 0.001

loss_multiplier = 10

learning_rate = desired_learning_rate / loss_multiplier

loss_function = fp16_loss_generator(tf.keras.losses.categorical_crossentropy, multiplier=loss_multiplier)
model = build_model(dtype)
model.compile(tf.train.RMSPropOptimizer(learning_rate), loss_function, metrics=['accuracy'])

TypeError: super(type, obj): obj must be an instance or subtype of type

In [14]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 32, 32, 32)   896         input_3[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_18 (LeakyReLU)      (None, 32, 32, 32)   0           conv2d_26[0][0]                  
__________________________________________________________________________________________________
conv2d_28 (Conv2D)              (None, 32, 32, 32)   9248        leaky_re_lu_18[0][0]             
__________________________________________________________________________________________________
lambda_32 

In [16]:
model.fit(x_train, y_train, batch_size=256, epochs=5)

Epoch 1/5
50000/50000 [==============================] - 11s 213us/step - loss: 7.5310 - acc: 0.7391
Epoch 2/5
50000/50000 [==============================] - 11s 215us/step - loss: 6.8878 - acc: 0.7611
Epoch 3/5
50000/50000 [==============================] - 11s 215us/step - loss: 6.3023 - acc: 0.7809
Epoch 4/5
50000/50000 [==============================] - 11s 214us/step - loss: 5.7573 - acc: 0.8018
Epoch 5/5
50000/50000 [==============================] - 11s 214us/step - loss: 5.2574 - acc: 0.8205


In [1]:
model.evaluate(x_test, y_test)

NameError: name 'model' is not defined

In [ ]:
tf.keras.backend.set_floatx('float32')
tf.keras.backend.set_epsilon(1e-7)
dtype = tf.float32

model = build_model(dtype)
model.compile(tf.train.RMSPropOptimizer(0.001, decay=1e-6), tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [11]:
model.fit(x_train, y_train, batch_size=256, epochs=5)


Epoch 1/5
50000/50000 [==============================] - 14s 272us/step - loss: 1.5763 - acc: 0.4691
Epoch 2/5
50000/50000 [==============================] - 11s 217us/step - loss: 1.0231 - acc: 0.6692
Epoch 3/5
50000/50000 [==============================] - 11s 219us/step - loss: 0.8502 - acc: 0.7360
Epoch 4/5
50000/50000 [==============================] - 11s 214us/step - loss: 0.7391 - acc: 0.7811
Epoch 5/5
50000/50000 [==============================] - 11s 216us/step - loss: 0.6615 - acc: 0.8098


In [15]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 2s 183us/step


[1.1304197205543518, 0.6792]